In [ ]:
import os

from pyincore import IncoreClient, FragilityCurveSet, FragilityService
from pyincore.dataservice import DataService
from pyincore.hazardservice import HazardService
from pyincore import Dataset
from pyincore import NetworkDataset
from pyincore_viz.geoutil import GeoUtil as viz
from pyincore_viz.plotutil import PlotUtil as plot

In [ ]:
# Connect to IN-CORE Services
client = IncoreClient()

In [ ]:
# testing datasets 

tornado_hazard_id = "5d07cbeab9219c065b080930"

joplin_bldg_inv_id = "5dbc8478b9219c06dd242c0d"

eq_hazard_id = "5b902cb273c3371e1236b36b"

shelby_hospital_inv_id = "5a284f0bc7d30d13bc081a28"

shelby_road_id = "5a284f2bc7d30d13bc081eb6"

centerville_epn_network_id = "5f454c6fef0df52132b65b0b"

galvaston_wave_height_id = '5f15cd62c98cf43417c10a3f'

memphis_water_buried_pipeline = "5a284f28c7d30d13bc081d14"

shelby_census_tract = "5a284f4cc7d30d13bc0822d4"

galveston_deterministic_hurricane = "5f15cd62c98cf43417c10a3f"

In [ ]:
eq_metadata  = HazardService(client).get_earthquake_hazard_metadata(eq_hazard_id)
eq_dataset_id = eq_metadata['rasterDataset']['datasetId']

eq_dataset = Dataset.from_data_service(eq_dataset_id, DataService(client))

In [ ]:
viz.plot_earthquake(eq_hazard_id, client)

In [ ]:
viz.plot_raster_dataset(galveston_deterministic_hurricane, client)

In [ ]:
sh_bldg_inv = Dataset.from_data_service(shelby_hospital_inv_id, DataService(client))

In [ ]:
sh_road = Dataset.from_data_service(shelby_road_id, DataService(client))

In [ ]:
viz.plot_map(sh_bldg_inv, column="struct_typ", category=False, basemap=True)

In [ ]:
viz.get_wms_map([sh_bldg_inv, sh_road])

In [ ]:
viz.get_gdf_map([sh_bldg_inv, sh_road])

In [ ]:
viz.get_gdf_wms_map([sh_bldg_inv], [sh_road])

In [ ]:
viz.plot_tornado(tornado_hazard_id, client, basemap=False)

In [ ]:
tornado_dataset_id = HazardService(client).get_tornado_hazard_metadata(tornado_hazard_id)['datasetId']
tornado_dataset = Dataset.from_data_service(tornado_dataset_id, DataService(client))

In [ ]:
viz.get_gdf_map([tornado_dataset])

In [ ]:
joplin_bldg_inv = Dataset.from_data_service(joplin_bldg_inv_id, DataService(client))

In [ ]:
# using wms layer for joplin building inv. gdf will crash the browser
viz.get_gdf_wms_map([tornado_dataset], [joplin_bldg_inv])

In [ ]:
# visualize incore network dataset
dataset = Dataset.from_data_service(centerville_epn_network_id, DataService(client))
network_dataset = NetworkDataset(dataset)
viz.plot_network_dataset(network_dataset)

In [ ]:
# visualize geotiff
dataset = Dataset.from_data_service(galvaston_wave_height_id, DataService(client))
map = viz.plot_map_raster_from_path(dataset.get_file_path('tif'))
map

In [ ]:
# add opacity control
map.layers[1].interact(opacity=(0.0,1.0,0.01))

In [ ]:
# visualize point heatmap
dataset = Dataset.from_data_service(shelby_hospital_inv_id, DataService(client))
map = viz.plot_heatmap(dataset, "str_prob", radius=40, blur=50, max=0.1, multiplier=100)
map

In [ ]:
# visualize line heatmap
dataset = Dataset.from_data_service(memphis_water_buried_pipeline, DataService(client))
map = viz.plot_heatmap(dataset, "length", radius=30, blur=50, max=0.5, multiplier=100)
map

In [ ]:
# visualize polygon heatmap
dataset = Dataset.from_data_service(shelby_census_tract, DataService(client))
map = viz.plot_heatmap(dataset, "id", radius=30, blur=50, max=0.1)
map

In [ ]:
# visualize 2d new format fragility curves
fragility_set = FragilityCurveSet(FragilityService(client).get_dfr3_set("602f31f381bd2c09ad8efcb4"))
plt = plot.get_fragility_plot_2d_refactored(fragility_set, title="2D visualization of fragility curve")
plt.show()

In [ ]:
# configure custom fragility curve parameters
custom_fragility_curve_parameters={"ffe_elev": 5}
fragility_set = FragilityCurveSet(FragilityService(client).get_dfr3_set("602f31f381bd2c09ad8efcb4"))
plt = plot.get_fragility_plot_2d_refactored(fragility_set, 
                                            title="2D visualization of fragility curve (first floor elevation 5m)", 
                                            custom_fragility_curve_parameters=custom_fragility_curve_parameters)
plt.show()

In [ ]:
# visualize format 3d
fragility_set = FragilityCurveSet(FragilityService(client).get_dfr3_set("5f6ccf67de7b566bb71b202d"))
plt = plot.get_fragility_plot_3d_refactored(fragility_set, title="3D visualization of fragility curve",
                                            limit_state="LS_0")
plt.show()